# LangChain 0.3 结构化输出详解
> 结构化输出是 LangChain 0.3 的核心功能，让 LLM 返回符合预定义格式的数据结构，而不是纯文本。这对于构建可靠的 AI 应用至关重要。

核心概念
LangChain 0.3 提供了两种主要的结构化输出方式：
1. .withStructuredOutput() - 推荐方式，简单易用
2. StructuredOutputParser - 传统方式，更灵活

## 1. 基础结构化输出 - withStructuredOutput()

In [1]:
# 1. 基础结构化输出示例
from langchain_ollama import ChatOllama
from pydantic import BaseModel, Field
from typing import List, Optional

# 定义数据结构
class Person(BaseModel):
    """人物信息结构"""
    name: str = Field(description="人物姓名")
    age: int = Field(description="年龄")
    occupation: str = Field(description="职业")
    skills: List[str] = Field(description="技能列表")
    location: Optional[str] = Field(description="居住地", default=None)

def basic_structured_output():
    """基础结构化输出示例"""
    print("=" * 50)
    print("1. 基础结构化输出")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 使用 withStructuredOutput 绑定结构
    structured_model = model.with_structured_output(Person)

    # 直接调用，返回结构化对象
    result = structured_model.invoke("请介绍一下马云的基本信息")

    print(f"结果类型: {type(result)}")
    print(f"姓名: {result.name}")
    print(f"年龄: {result.age}")
    print(f"职业: {result.occupation}")
    print(f"技能: {result.skills}")
    print(f"居住地: {result.location}")

    return result

basic_structured_output()

1. 基础结构化输出
结果类型: <class '__main__.Person'>
姓名: 马云
年龄: 65
职业: 前阿里巴巴董事局主席、淘宝网创始人
技能: ['企业家', '创业者', '公共演讲家', '社会活动家']
居住地: 中国杭州


Person(name='马云', age=65, occupation='前阿里巴巴董事局主席、淘宝网创始人', skills=['企业家', '创业者', '公共演讲家', '社会活动家'], location='中国杭州')

## 2. 复杂嵌套结构

In [2]:
# 2. 复杂嵌套结构示例
from datetime import datetime
from typing import Dict, Any

class Address(BaseModel):
    """地址信息"""
    city: str = Field(description="城市")
    country: str = Field(description="国家")
    postal_code: Optional[str] = Field(description="邮政编码", default=None)

class Employee(BaseModel):
    """员工信息"""
    name: str = Field(description="员工姓名")
    position: str = Field(description="职位")
    department: str = Field(description="部门")
    salary: Optional[int] = Field(description="薪资", default=None)

class Revenue(BaseModel):
    """收入信息"""
    amount: float = Field(description="收入金额")
    currency: str = Field(description="货币单位")
    year: int = Field(description="年份")

class Company(BaseModel):
    """公司信息结构"""
    name: str = Field(description="公司名称")
    founded: int = Field(description="成立年份")
    headquarters: Address = Field(description="总部信息")
    employees: List[Employee] = Field(description="关键员工列表")
    revenue: Revenue = Field(description="最新收入信息")
    products: List[str] = Field(description="主要产品列表")

def nested_structured_output():
    """复杂嵌套结构示例"""
    print("\n" + "=" * 50)
    print("2. 复杂嵌套结构")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    structured_model = model.with_structured_output(Company)

    result = structured_model.invoke("""
    请提供阿里巴巴集团的详细信息，包括：
    - 公司基本信息和总部地址
    - 3-5个关键员工（包括马云、张勇等）
    - 最新年度收入数据
    - 主要产品和服务
    """)

    print(f"公司名称: {result.name}")
    print(f"成立年份: {result.founded}")
    print(f"总部: {result.headquarters.city}, {result.headquarters.country}")
    print(f"员工数量: {len(result.employees)}")
    print("关键员工:")
    for emp in result.employees:
        print(f"  - {emp.name}: {emp.position} ({emp.department})")
    print(f"收入: {result.revenue.amount} {result.revenue.currency} ({result.revenue.year})")
    print(f"主要产品: {', '.join(result.products)}")

    return result

nested_structured_output()


2. 复杂嵌套结构
公司名称: 阿里巴巴集团
成立年份: 1999
总部: 杭州, 中国
员工数量: 4
关键员工:
  - 马云: 创始人，前董事局主席 ()
  - 张勇: 董事会成员,CEO (阿里巴巴集团)
  - 蔡崇信: 董事总经理 (阿里巴巴集团)
  - 井贤栋: 副董事长 (蚂蚁金服)
收入: 5304.6 Billion US Dollar (2021)
主要产品: 天猫, B站, 盒马, 菜鸟网络, 阿里云, 高德地图, 优酷


Company(name='阿里巴巴集团', founded=1999, headquarters=Address(city='杭州', country='中国', postal_code=None), employees=[Employee(name='马云', position='创始人，前董事局主席', department='', salary=None), Employee(name='张勇', position='董事会成员,CEO', department='阿里巴巴集团', salary=None), Employee(name='蔡崇信', position='董事总经理', department='阿里巴巴集团', salary=None), Employee(name='井贤栋', position='副董事长', department='蚂蚁金服', salary=None)], revenue=Revenue(amount=5304.6, currency='Billion US Dollar', year=2021), products=['天猫', 'B站', '盒马', '菜鸟网络', '阿里云', '高德地图', '优酷'])

## 3. 使用 StructuredOutputParser

In [3]:
# 3. StructuredOutputParser 示例
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate

class BookReview(BaseModel):
    """书籍评价结构"""
    title: str = Field(description="书籍标题")
    author: str = Field(description="作者")
    rating: int = Field(description="评分(1-5)", ge=1, le=5)
    summary: str = Field(description="内容摘要")
    pros: List[str] = Field(description="优点列表")
    cons: List[str] = Field(description="缺点列表")
    recommendation: bool = Field(description="是否推荐")

def structured_output_parser():
    """StructuredOutputParser 示例"""
    print("\n" + "=" * 50)
    print("3. StructuredOutputParser")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 创建解析器
    parser = PydanticOutputParser(pydantic_object=BookReview)

    # 创建提示模板
    prompt = ChatPromptTemplate.from_template("""
    请对以下书籍进行详细评价：{book}

    {format_instructions}

    请确保返回有效的JSON格式。
    """)

    # 创建链
    chain = prompt | model | parser

    # 执行
    result = chain.invoke({
        "book": "《三体》刘慈欣",
        "format_instructions": parser.get_format_instructions()
    })

    print(f"书籍: {result.title}")
    print(f"作者: {result.author}")
    print(f"评分: {result.rating}/5")
    print(f"摘要: {result.summary}")
    print(f"优点: {', '.join(result.pros)}")
    print(f"缺点: {', '.join(result.cons)}")
    print(f"推荐: {'是' if result.recommendation else '否'}")

    return result

structured_output_parser()


3. StructuredOutputParser
书籍: 三体
作者: 刘慈欣
评分: 5/5
摘要: 《三体》是刘慈欣的科幻巨著，讲述了地球文明和三体文明的冲突与融合。通过宏大的叙事结构和深入的科学探讨，《三体》不仅是一部科幻小说，更是一场关于人类命运、宇宙奥秘的大辩论。
优点: 独特的世界观构建, 深刻的人文关怀, 创新的科学理论
缺点: 长篇累牍不易阅读
推荐: 是


BookReview(title='三体', author='刘慈欣', rating=5, summary='《三体》是刘慈欣的科幻巨著，讲述了地球文明和三体文明的冲突与融合。通过宏大的叙事结构和深入的科学探讨，《三体》不仅是一部科幻小说，更是一场关于人类命运、宇宙奥秘的大辩论。', pros=['独特的世界观构建', '深刻的人文关怀', '创新的科学理论'], cons=['长篇累牍不易阅读'], recommendation=True)

## 4. 批量结构化输出

In [4]:
# 4. 批量结构化输出
class Product(BaseModel):
    """产品信息结构"""
    name: str = Field(description="产品名称")
    category: str = Field(description="产品类别")
    price: float = Field(description="价格(美元)")
    features: List[str] = Field(description="主要特性")
    target_audience: str = Field(description="目标用户群体")
    pros: List[str] = Field(description="优势")
    cons: List[str] = Field(description="劣势")

def batch_structured_output():
    """批量结构化输出示例"""
    print("\n" + "=" * 50)
    print("4. 批量结构化输出")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    structured_model = model.with_structured_output(Product)

    products = [
        "iPhone 15 Pro",
        "Tesla Model 3",
        "ChatGPT Plus订阅",
        "Nintendo Switch"
    ]

    # 批量处理
    prompts = [f"请详细分析产品: {product}，包括价格、特性、目标用户等信息" for product in products]
    results = structured_model.batch(prompts)

    for i, result in enumerate(results):
        print(f"\n--- 产品 {i+1}: {products[i]} ---")
        print(f"名称: {result.name}")
        print(f"类别: {result.category}")
        print(f"价格: ${result.price}")
        print(f"特性: {', '.join(result.features[:3])}...")  # 只显示前3个特性
        print(f"目标用户: {result.target_audience}")

    return results

batch_structured_output()


4. 批量结构化输出

--- 产品 1: iPhone 15 Pro ---
名称: iPhone 15 Pro
类别: 智能手机
价格: $8999.0
特性: A16芯片, 4800万像素主摄镜头, ProMotion自适应刷新率显示技术...
目标用户: 高端用户,尤其是专业人士，如摄影师、视频制作者和科技爱好者

--- 产品 2: Tesla Model 3 ---
名称: Tesla Model 3
类别: Electric Vehicle
价格: $294.0
特性: Fast charging capability, Autopilot advanced features, Long driving range on a single charge...
目标用户: Tesla Model 3 is aimed at tech enthusiasts, young professionals, and individuals looking for a high-performance electric vehicle within their budget. It also appeals to families who want a stylish, efficient, and reliable car. Additionally, Tesla is targeting business customers with its leasing options.

--- 产品 3: ChatGPT Plus订阅 ---
名称: ChatGPT Plus
类别: 人工智能服务
价格: $20.0
特性: 1小时访问次数增加到20次, 包含24/7的客户支持, 无限制的API调用权限...
目标用户: 高级用户，专业团队和企业客户

--- 产品 4: Nintendo Switch ---
名称: Nintendo Switch
类别: 电子游戏设备
价格: $299.0
特性: 便携式设计：可以折叠到一个小背包中携带，非常适合去户外游玩。, 高性能处理器和流畅的游戏体验：配备有更快的Ampere架构GPU以及最新版本的Nintendo Switch OS。, 跨平台游戏支持：用户可以在Switch上玩到任天堂自家开发、微软Xbox或谷歌Sta

[Product(name='iPhone 15 Pro', category='智能手机', price=8999.0, features=['A16芯片', '4800万像素主摄镜头', 'ProMotion自适应刷新率显示技术', 'MagSafe磁吸充电接口', 'IP6防水防尘设计', '新一代超广角摄像头', '智能HDR 5影像系统', 'S Pen手写笔兼容性', '增强的Apple Vision Pro集成支持', '先进的隐私保护功能', '更长的电池续航能力'], target_audience='高端用户,尤其是专业人士，如摄影师、视频制作者和科技爱好者', pros=['强大的性能和高效的图像处理速度', '出色的摄影能力', '稳定的自适应刷新率显示技术', '独特的MagSafe充电设计', '先进的隐私保护功能', '持久的电池寿命'], cons=['较高价格']),
 Product(name='Tesla Model 3', category='Electric Vehicle', price=294.0, features=['Fast charging capability', 'Autopilot advanced features', 'Long driving range on a single charge', 'Driverless technology (with Autopilot)', "Seamless connectivity with Tesla's ecosystem", "Semi-autonomous driving capabilities through 'Full Self-Driving' Beta"], target_audience='Tesla Model 3 is aimed at tech enthusiasts, young professionals, and individuals looking for a high-performance electric vehicle within their budget. It also appeals to families who want a stylish, efficient, and reliable car. Ad

## 5. 条件结构化输出

In [5]:
# 5. 条件结构化输出
from typing import Union, Literal

class PersonInfo(BaseModel):
    """人物信息"""
    type: Literal["person"] = "person"
    name: str = Field(description="姓名")
    age: int = Field(description="年龄")
    occupation: str = Field(description="职业")

class CompanyInfo(BaseModel):
    """公司信息"""
    type: Literal["company"] = "company"
    name: str = Field(description="公司名称")
    founded: int = Field(description="成立年份")
    industry: str = Field(description="行业")

class EventInfo(BaseModel):
    """事件信息"""
    type: Literal["event"] = "event"
    name: str = Field(description="事件名称")
    date: str = Field(description="日期")
    location: str = Field(description="地点")

# 联合类型
EntityInfo = Union[PersonInfo, CompanyInfo, EventInfo]

def conditional_structured_output():
    """条件结构化输出示例"""
    print("\n" + "=" * 50)
    print("5. 条件结构化输出")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 创建解析器处理联合类型
    parser = PydanticOutputParser(pydantic_object=EntityInfo)

    prompt = ChatPromptTemplate.from_template("""
    请分析以下内容并提取相关信息：{content}

    根据内容类型返回对应的结构：
    - 如果是人物，返回 person 类型
    - 如果是公司，返回 company 类型
    - 如果是事件，返回 event 类型

    {format_instructions}
    """)

    chain = prompt | model | parser

    test_cases = [
        "马云，1964年出生，阿里巴巴创始人",
        "苹果公司成立于1976年，是一家科技公司",
        "2024年巴黎奥运会将在法国巴黎举行"
    ]

    for content in test_cases:
        print(f"\n输入: {content}")
        result = chain.invoke({
            "content": content,
            "format_instructions": parser.get_format_instructions()
        })

        print(f"类型: {result.type}")
        if result.type == "person":
            print(f"姓名: {result.name}, 年龄: {result.age}, 职业: {result.occupation}")
        elif result.type == "company":
            print(f"公司: {result.name}, 成立: {result.founded}, 行业: {result.industry}")
        elif result.type == "event":
            print(f"事件: {result.name}, 日期: {result.date}, 地点: {result.location}")

conditional_structured_output()


5. 条件结构化输出

输入: 马云，1964年出生，阿里巴巴创始人


AttributeError: model_json_schema

## 6. 流式结构化输出

In [6]:
# 6. 流式结构化输出
import asyncio
from langchain_core.runnables import RunnablePassthrough

class NewsAnalysis(BaseModel):
    """新闻分析结构"""
    headline: str = Field(description="新闻标题")
    summary: str = Field(description="新闻摘要")
    category: str = Field(description="新闻类别")
    keywords: List[str] = Field(description="关键词")
    sentiment: Literal["positive", "negative", "neutral"] = Field(description="情感倾向")
    importance: int = Field(description="重要性评分(1-10)", ge=1, le=10)

async def streaming_structured_output():
    """流式结构化输出示例"""
    print("\n" + "=" * 50)
    print("6. 流式结构化输出")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    structured_model = model.with_structured_output(NewsAnalysis)

    news_content = """
    中国成功发射神舟十八号载人飞船，三名航天员顺利进入太空站。
    这标志着中国载人航天事业又迈出重要一步，展现了中国在航天领域的技术实力。
    """

    print("开始流式生成新闻分析...")
    print("输入新闻:", news_content.strip())

    try:
        # 注意：某些模型可能不支持结构化输出的流式处理
        # 这里我们使用普通调用作为示例
        result = await structured_model.ainvoke(f"请分析这条新闻：{news_content}")

        print(f"\n标题: {result.headline}")
        print(f"摘要: {result.summary}")
        print(f"类别: {result.category}")
        print(f"关键词: {', '.join(result.keywords)}")
        print(f"情感: {result.sentiment}")
        print(f"重要性: {result.importance}/10")

        return result
    except Exception as e:
        print(f"流式处理出错: {e}")
        # 降级到普通调用
        result = structured_model.invoke(f"请分析这条新闻：{news_content}")
        print("使用普通调用完成分析")
        return result

# 运行异步函数
import nest_asyncio
nest_asyncio.apply()
asyncio.run(streaming_structured_output())


6. 流式结构化输出
开始流式生成新闻分析...
输入新闻: 中国成功发射神舟十八号载人飞船，三名航天员顺利进入太空站。
    这标志着中国载人航天事业又迈出重要一步，展现了中国在航天领域的技术实力。

标题: 中国成功发射神舟十八号载人飞船
摘要: 该新闻报道了中国成功发射了名为‘神舟十八号’的载人飞船，并且三名航天员已顺利进入太空站。这标志着中国在载人航天领域取得了重要进展，进一步证明了中国在相关领域的技术实力。
类别: 科技/航天
关键词: 神舟十八号, 载人飞船, 三名航天员, 太空站, 中国载人航天事业, 技术水平
情感: positive
重要性: 1/10


NewsAnalysis(headline='中国成功发射神舟十八号载人飞船', summary='该新闻报道了中国成功发射了名为‘神舟十八号’的载人飞船，并且三名航天员已顺利进入太空站。这标志着中国在载人航天领域取得了重要进展，进一步证明了中国在相关领域的技术实力。', category='科技/航天', keywords=['神舟十八号', '载人飞船', '三名航天员', '太空站', '中国载人航天事业', '技术水平'], sentiment='positive', importance=1)

## 7. 错误处理和验证

In [ ]:
# 7. 错误处理和验证
from pydantic import validator, ValidationError
from langchain_core.output_parsers import OutputParserException

class ValidatedProduct(BaseModel):
    """带验证的产品信息"""
    name: str = Field(description="产品名称")
    price: float = Field(description="价格", gt=0)  # 必须大于0
    category: str = Field(description="产品类别")
    rating: float = Field(description="评分", ge=0, le=5)  # 0-5之间

    @validator('name')
    def name_must_not_be_empty(cls, v):
        if not v.strip():
            raise ValueError('产品名称不能为空')
        return v.strip()

    @validator('category')
    def category_must_be_valid(cls, v):
        valid_categories = ['电子产品', '服装', '食品', '书籍', '其他']
        if v not in valid_categories:
            raise ValueError(f'类别必须是: {", ".join(valid_categories)}')
        return v

def error_handling_example():
    """错误处理示例"""
    print("\n" + "=" * 50)
    print("7. 错误处理和验证")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    parser = PydanticOutputParser(pydantic_object=ValidatedProduct)

    prompt = ChatPromptTemplate.from_template("""
    请分析以下产品信息：{product}

    {format_instructions}

    注意：
    - 价格必须大于0
    - 评分必须在0-5之间
    - 类别必须是：电子产品、服装、食品、书籍、其他 之一
    """)

    chain = prompt | model | parser

    test_products = [
        "iPhone 15 Pro - 高端智能手机",  # 正常情况
        "免费软件 - 价格为0的产品",      # 价格验证失败
        "神秘产品 - 未知类别"           # 类别验证失败
    ]

    for product in test_products:
        print(f"\n测试产品: {product}")
        try:
            result = chain.invoke({
                "product": product,
                "format_instructions": parser.get_format_instructions()
            })
            print("✅ 解析成功:")
            print(f"  名称: {result.name}")
            print(f"  价格: ${result.price}")
            print(f"  类别: {result.category}")
            print(f"  评分: {result.rating}/5")

        except ValidationError as e:
            print("❌ 验证失败:")
            for error in e.errors():
                print(f"  {error['loc'][0]}: {error['msg']}")

        except OutputParserException as e:
            print(f"❌ 解析失败: {e}")

        except Exception as e:
            print(f"❌ 其他错误: {e}")

error_handling_example()

## 8. 与 LCEL 链式组合

In [ ]:
# 8. 与 LCEL 链式组合
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

class TextAnalysis(BaseModel):
    """文本分析结构"""
    language: str = Field(description="语言")
    word_count: int = Field(description="词数")
    sentiment: Literal["positive", "negative", "neutral"] = Field(description="情感")
    topics: List[str] = Field(description="主题列表")

class Translation(BaseModel):
    """翻译结构"""
    original_text: str = Field(description="原文")
    translated_text: str = Field(description="译文")
    source_language: str = Field(description="源语言")
    target_language: str = Field(description="目标语言")

def lcel_chain_example():
    """LCEL 链式组合示例"""
    print("\n" + "=" * 50)
    print("8. LCEL 链式组合")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    # 创建分析和翻译模型
    analysis_model = model.with_structured_output(TextAnalysis)
    translation_model = model.with_structured_output(Translation)

    # 创建并行处理链
    parallel_chain = RunnableParallel({
        "analysis": analysis_model,
        "translation": translation_model,
        "original": RunnablePassthrough()
    })

    # 创建提示模板
    analysis_prompt = ChatPromptTemplate.from_template("请分析这段文本：{text}")
    translation_prompt = ChatPromptTemplate.from_template("请将这段中文翻译成英文：{text}")

    # 组合完整链
    full_chain = RunnableParallel({
        "analysis": analysis_prompt | analysis_model,
        "translation": translation_prompt | translation_model,
        "original": RunnablePassthrough()
    })

    test_text = "今天天气真好，我很开心能够学习LangChain的结构化输出功能。这个功能非常强大，可以让AI返回标准化的数据结构。"

    result = full_chain.invoke({"text": test_text})

    print("原文:", test_text)
    print("\n--- 文本分析 ---")
    analysis = result["analysis"]
    print(f"语言: {analysis.language}")
    print(f"词数: {analysis.word_count}")
    print(f"情感: {analysis.sentiment}")
    print(f"主题: {', '.join(analysis.topics)}")

    print("\n--- 翻译结果 ---")
    translation = result["translation"]
    print(f"源语言: {translation.source_language}")
    print(f"目标语言: {translation.target_language}")
    print(f"译文: {translation.translated_text}")

lcel_chain_example()

## 9. 实际应用场景

In [ ]:
# 9. 实际应用场景 - 简历解析系统
class Education(BaseModel):
    """教育经历"""
    degree: str = Field(description="学位")
    major: str = Field(description="专业")
    school: str = Field(description="学校")
    graduation_year: int = Field(description="毕业年份")

class WorkExperience(BaseModel):
    """工作经历"""
    company: str = Field(description="公司名称")
    position: str = Field(description="职位")
    duration: str = Field(description="工作时长")
    responsibilities: List[str] = Field(description="主要职责")

class Resume(BaseModel):
    """简历结构"""
    name: str = Field(description="姓名")
    email: str = Field(description="邮箱")
    phone: str = Field(description="电话")
    education: List[Education] = Field(description="教育经历")
    work_experience: List[WorkExperience] = Field(description="工作经历")
    skills: List[str] = Field(description="技能列表")
    summary: str = Field(description="个人总结")

def resume_parsing_example():
    """简历解析应用示例"""
    print("\n" + "=" * 50)
    print("9. 实际应用 - 简历解析")
    print("=" * 50)

    model = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen2.5:3b"
    )

    structured_model = model.with_structured_output(Resume)

    resume_text = """
    张三
    邮箱: zhangsan@email.com
    电话: 138-0000-0000

    教育背景：
    2018-2022 北京大学 计算机科学与技术 本科
    2022-2024 清华大学 人工智能 硕士

    工作经验：
    2024.3-至今 阿里巴巴 算法工程师
    - 负责推荐系统算法优化
    - 参与大模型训练项目
    - 提升用户点击率15%

    2023.6-2024.2 腾讯 实习生
    - 协助开发聊天机器人
    - 数据清洗和模型评估

    技能：
    Python, Java, TensorFlow, PyTorch, SQL, Git

    个人总结：
    热爱AI技术，有丰富的机器学习项目经验，善于团队合作。
    """

    print("解析简历...")
    result = structured_model.invoke(f"请解析以下简历信息：\n{resume_text}")

    print(f"\n姓名: {result.name}")
    print(f"邮箱: {result.email}")
    print(f"电话: {result.phone}")

    print("\n教育经历:")
    for edu in result.education:
        print(f"  {edu.graduation_year} {edu.school} {edu.major} {edu.degree}")

    print("\n工作经历:")
    for work in result.work_experience:
        print(f"  {work.company} - {work.position} ({work.duration})")
        for resp in work.responsibilities:
            print(f"    • {resp}")

    print(f"\n技能: {', '.join(result.skills)}")
    print(f"\n个人总结: {result.summary}")

    return result

resume_parsing_example()

## 10. 性能优化和最佳实践

In [7]:
# 10. 性能优化和最佳实践
import time
from functools import wraps

def timing_decorator(func):
    """计时装饰器"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"⏱️ {func.__name__} 耗时: {end - start:.2f}秒")
        return result
    return wrapper

class OptimizedProduct(BaseModel):
    """优化的产品结构 - 简化字段"""
    name: str
    price: float
    category: str

class DetailedProduct(BaseModel):
    """详细的产品结构 - 复杂字段"""
    name: str = Field(description="产品名称")
    price: float = Field(description="价格")
    category: str = Field(description="类别")
    description: str = Field(description="详细描述")
    features: List[str] = Field(description="特性列表")
    specifications: Dict[str, str] = Field(description="规格参数")
    reviews: List[str] = Field(description="用户评价")

@timing_decorator
def simple_structure_test():
    """简单结构性能测试"""
    model = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b")
    structured_model = model.with_structured_output(OptimizedProduct)

    result = structured_model.invoke("iPhone 15 Pro的基本信息")
    return result

@timing_decorator
def complex_structure_test():
    """复杂结构性能测试"""
    model = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:3b")
    structured_model = model.with_structured_output(DetailedProduct)

    result = structured_model.invoke("iPhone 15 Pro的详细信息，包括特性、规格、用户评价等")
    return result

def best_practices_demo():
    """最佳实践演示"""
    print("\n" + "=" * 50)
    print("10. 性能优化和最佳实践")
    print("=" * 50)

    print("🔍 性能对比测试:")

    print("\n1. 简单结构测试:")
    simple_result = simple_structure_test()
    print(f"结果: {simple_result.name} - ${simple_result.price}")

    print("\n2. 复杂结构测试:")
    complex_result = complex_structure_test()
    print(f"结果: {complex_result.name} - ${complex_result.price}")
    print(f"特性数量: {len(complex_result.features)}")

    print("\n📋 最佳实践建议:")
    print("✅ 1. 优先使用 .with_structured_output() 方法")
    print("✅ 2. 保持结构简单，避免过度嵌套")
    print("✅ 3. 使用清晰的字段描述")
    print("✅ 4. 合理使用 Optional 字段")
    print("✅ 5. 添加数据验证规则")
    print("✅ 6. 批量处理相似任务")
    print("✅ 7. 适当的错误处理")
    print("✅ 8. 缓存常用的结构化模型")

best_practices_demo()


10. 性能优化和最佳实践
🔍 性能对比测试:

1. 简单结构测试:
⏱️ simple_structure_test 耗时: 4.06秒
结果: iPhone 15 Pro - $999.0

2. 复杂结构测试:


KeyboardInterrupt: 

# 总结
1. 基础用法 - .with_structured_output() 方法
2. 复杂结构 - 嵌套对象和列表
3. 传统方法 - StructuredOutputParser
4. 批量处理 - 提高效率
5. 条件输出 - 联合类型处理
6. 流式输出 - 实时处理
7. 错误处理 - 数据验证
8. 链式组合 - 与 LCEL 结合
9. 实际应用 - 简历解析系统
10. 性能优化 - 最佳实践